# Custom Models, Layers, and Loss Functions with TensorFlow 
### *Week: 2*

#### DeepLearningAI

## Description

* Compare Functional and Sequential APIs, discover new models you can build with the Functional API, and build a model that produces multiple outputs including a Siamese network.
* Build custom loss functions (including the contrastive loss function used in a Siamese network) in order to measure how well a model is doing and help your neural network learn from training data. 
* Build off of existing standard layers to create custom layers for your models, customize a network layer with a lambda layer, understand the differences between them, learn what makes up a custom layer, and explore activation functions. 
* Build off of existing models to add custom functionality, learn how to define your own custom class instead of using the Functional or Sequential APIs, build models that can be inherited from the TensorFlow Model class, and build a residual network (ResNet) through defining a custom model class. 


The DeepLearning.AI TensorFlow: Advanced Techniques Specialization introduces the features of TensorFlow that provide learners with more control over their model architecture and tools that help them create and train advanced ML models.  

This Specialization is for early and mid-career software and machine learning engineers with a foundational understanding of TensorFlow who are looking to expand their knowledge and skill set by learning advanced TensorFlow features to build powerful models.

## Custom Loss Functions

### Using Loss Functions

```Python
model.compile(loss='mse', optimizer='sgd')

# OR

from tensorflow.keras.losses import mean_squared_error
model.compile(loss=mean_squared_error(param=value), optimizer='sgd')
```

* We could use parameters in loss functions if loss functions is a object.

### Creating a custom loss function

```Python
def my_loss_function(y_true, y_pred):
    ...
    return losses
```

* *y_true* will contain the labels. -The source of truth.
* *y_pred* is predicted / calculated value in network.

![HuberLossFunction](huber-loss.png)

##### [Reference](https://en.wikipedia.org/wiki/Huber_loss)

### Coding the Huber Loss Function

```Python
def my_huber_loss(y_true, y_pred):
    threshold = 1
    error = y_true - y_pred
    is_small_error = tf.abs(error) <= threshold
    small_error_loss = tf.square(error) / 2
    big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
    return tf.where(is_small_error, smal_error_loss, big_error_loss)
```

*Example*

```Python
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss='my_huber_loss')
```

### [The first lab notebook](C1_W2_Lab_1_huber-loss.ipynb)

## Custom Loss Hyperparameters and Classes

### Adding Hyperparameters to Custom Loss Functions

The *threshold* variable will be the parameter which will come outside thanks to the **wrapper function**.

```Python
def my_huber_loss_with_threshold(threshold):
    def my_huber_loss(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) <= threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
        return tf.where(is_small_error, small_error_loss, big_error_loss)
    return my_huber_los


model.compile(optimizer='sgd', loss=my_huber_loss_with_threshold(threshold=1)) # Sending parameter
```

### Turning loss functions into Classes

```Python
from tensorflow.keras.losses import Loss

class MyHuberLoss(Loss):
    threshold = 1
    
    def __init__(self, threshold):
        super().__init__()
        self.threshold = threshold
        
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) <= self.threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = self.threshold * (tf.abs(error) - (0.5 * self.threshold))
        return tf.where(is_small_error, small_error_loss, big_error_loss)
    
## End    

model.compile(optimizer='sgd', loss=MyHuberLoss(threshold=1))
```

### [The second lab notebook](C1_W2_Lab_2_huber-object-loss.ipynb)

## Contrastive Loss

* If images are *similar*, produce feature vectors that are **very similar**.
* If images are *different*, produce feature vectors that are **dissimilar**.
* Based on the paper

>"Dimensionality Reduction by Learning an Invariant Mapping"
>-by R. Hadsell; S. Chopra; Y. LeCun

##### [Reference](http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf)


**Contrastive Loss - Formula** 
\\[Y * D^2 + (1 - Y) * max(margin - D, O)^2\\]

\\[Y_{true} * Y_{pred}^2 + (1 - Y_{true}) * max(margin - Y_{pred}, O)^2\\]

### Coding Contrastive Loss

```Python
# Custom Loss Function

def contrastive_loss(y_true, y_pred):
    magin=1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)


# Usage of Custom Loss

model.compile(loss=contrastive_loss, optimizer=RMSprop())


# Custom Loss Function with Arguments

def contrastive_loss_with_margin(margin):
    def contrastive_loss(y_true, y_pred):
        square_pred = K.square(y_pred)
        margin_square = K.square(K.maximum(margin - y_pred, 0))
        return K.mean(y_true * square_pred + (1 - y_true) * margin_square)
    return contrastive_loss


# Usage of Wrapper Loss Function

model.compile(loss=contrastive_loss_with_margin(margin=0.6), optimizer=rms)
```

#### OOP

```Python
class ContrastiveLoss(Loss):
    margin = 0
    def __init__(self, margin):
        super().__init__()
        self.margin = margin
        
    def call(self, y_true, y_pred):
        square_pred = K.square(y_pred)
        margin_square = K.square(K.maximum(self.margin - y_pred, 0))
        return K.mean(y_true * square_pred + (1 - y_true) * margin_square)
    
# Usage of OOP Loss
model.compile(loss=ContrastiveLoss(margin=1), optimizer=rms)
    
```